## 1. Preliminary

### 1.1 Context

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par les agents de la ville en 2016. Voici [les données](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P4/2016_Building_Energy_Benchmarking.csv) et [leur source](https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy). Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, **vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation** pour lesquels elles n’ont pas encore été mesurées.

<div class="alert alert-block alert-info">
Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)
</div>

Vous cherchez également à **évaluer l’intérêt de l’"[ENERGY STAR Score](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what)" pour la prédiction d’émissions**, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.
Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :
 1) Réaliser une courte analyse exploratoire.
 2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Avant de quitter la salle de brief, Douglas, le project lead, vous donne quelques pistes et erreurs à éviter :

> Douglas : L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées.. 
Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).
Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.


### 1.2 Requirements

In [1]:
package_list = ("pandas", "numpy", "matplotlib", "seaborn", "scikit-learn", "mlflow")

In [2]:
!python3 -V

Python 3.10.13


In [3]:
txt = !python3 -m pip freeze
check = lambda i: any([(pack in i) for pack in package_list])
txt = [i for i in txt if check(i)]
txt

['matplotlib==3.8.2',
 'matplotlib-inline==0.1.6',
 'mlflow==2.11.3',
 'numpy @ file:///Users/runner/miniforge3/conda-bld/numpy_1704280780572/work/dist/numpy-1.26.3-cp310-cp310-macosx_11_0_arm64.whl#sha256=f96d0b051b72345dbc317d793b2b34c7c4b7f41b0b791ffc93e820c45ba6a91c',
 'pandas==2.2.0',
 'scikit-learn==1.4.0',
 'seaborn==0.13.2']

### 1.3 Imports

In [4]:
# built in
import os, warnings
import time

# data
import pandas as pd
import numpy as np

# metrics
from sklearn.metrics import mean_squared_error, r2_score

# estimators
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, Perceptron
from sklearn.neural_network import MLPRegressor

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

## mlflow
import mlflow
import mlflow.sklearn

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# exceptions
from sklearn.exceptions import ConvergenceWarning

# pipeline et preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer



### 1.4 Graphics and option

In [5]:
# warnings.filterwarnings('ignore)
warnings.filterwarnings(action='once')

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module='_distutils_hack')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='importlib')

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

<div class="alert alert-block alert-info">
We disable the warnings.</div>

### 1.5 Data

In [6]:
# !tree

In [7]:
# os.listdir()

In [8]:
path="./data/cleaned/"
filename="df_TotalGHGEmissions_0.csv"

In [9]:
df = pd.read_csv(path+filename)
df.head()

,Neighborhood,GroupedPrimaryPropertyTypes,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding(s),ENERGYSTARScore,DistanceToDowntown,Log_TotalGHGEmissions
0,DOWNTOWN,Bâtiments d'Hébergement,1927.0,1.0,12.0,88434.0,60.0,0.864611,5.525373
1,DOWNTOWN,Bâtiments d'Hébergement,1996.0,1.0,11.0,88502.0,61.0,0.907278,5.693261
2,DOWNTOWN,Bâtiments d'Hébergement,1969.0,1.0,41.0,759392.0,43.0,1.047606,7.645053
3,DOWNTOWN,Bâtiments d'Hébergement,1926.0,1.0,10.0,61320.0,56.0,1.038057,5.660979
4,DOWNTOWN,Bâtiments d'Hébergement,1980.0,1.0,18.0,113580.0,75.0,1.100255,6.226556


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379 entries, 0 to 1378
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Neighborhood                 1379 non-null   object 
 1   GroupedPrimaryPropertyTypes  1379 non-null   object 
 2   YearBuilt                    1379 non-null   float64
 3   NumberofBuildings            1379 non-null   float64
 4   NumberofFloors               1379 non-null   float64
 5   PropertyGFABuilding(s)       1379 non-null   float64
 6   ENERGYSTARScore              924 non-null    float64
 7   DistanceToDowntown           1379 non-null   float64
 8   Log_TotalGHGEmissions        1379 non-null   float64
dtypes: float64(7), object(2)
memory usage: 97.1+ KB


## 2. Data Preparation

### 2.1 X & y

In [11]:
X = df.drop(columns="Log_TotalGHGEmissions")
y = df['Log_TotalGHGEmissions']

In [12]:
X.head()

,Neighborhood,GroupedPrimaryPropertyTypes,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding(s),ENERGYSTARScore,DistanceToDowntown
0,DOWNTOWN,Bâtiments d'Hébergement,1927.0,1.0,12.0,88434.0,60.0,0.864611
1,DOWNTOWN,Bâtiments d'Hébergement,1996.0,1.0,11.0,88502.0,61.0,0.907278
2,DOWNTOWN,Bâtiments d'Hébergement,1969.0,1.0,41.0,759392.0,43.0,1.047606
3,DOWNTOWN,Bâtiments d'Hébergement,1926.0,1.0,10.0,61320.0,56.0,1.038057
4,DOWNTOWN,Bâtiments d'Hébergement,1980.0,1.0,18.0,113580.0,75.0,1.100255


In [13]:
y.head()

0    5.525373
1    5.693261
2    7.645053
3    5.660979
4    6.226556
Name: Log_TotalGHGEmissions, dtype: float64

## 3. Models for different k-values (knn imputation)

In [14]:
# Configurer MLflow
mlflow.set_experiment('Gradient_Boosting_gas_k_val')

# Initialiser le DataFrame des résultats
results_df = pd.DataFrame(columns=['Model Name', 'Best Parameters', 'MSE Train', 'MSE Test', 'R2 Train', 'R2 Test', 'Best CV R2 Test', 'Fit Time'])

def train_eval_model(model, X_train, y_train, X_test, y_test, model_name, best_params=None, best_cv_score=None):
    # Set model parameters if provided
    if best_params:
        model.set_params(**best_params)

    # Mesurer le temps de début
    start_time = time.time()
    # Entraîner le modèle
    model.fit(X_train, y_train)
    # Mesurer le temps de fin et calculer le temps de fitting
    fit_time = time.time() - start_time

    # Prédictions sur les données d'entraînement et de test
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcul des métriques de performance
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Enregistrement de la performance du modèle avec MLflow
    with mlflow.start_run(run_name=f"{model_name}"):
        if best_params:
            mlflow.log_params(best_params)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("mse_test", mse_test)
        mlflow.log_metric("r2_train", r2_train)
        mlflow.log_metric("r2_test", r2_test)
        if best_cv_score is not None:
            mlflow.log_metric("best_cv_r2_test", best_cv_score)
        mlflow.sklearn.log_model(model, f"model_{model_name}")

        print(f"{model_name} - MSE Train: {mse_train}, MSE Test: {mse_test}, R2 Train: {r2_train}, R2 Test: {r2_test}")

    # # Plotting the predicted vs real values
    # plt.figure(figsize=(10, 6))
    # plt.scatter(y_test, y_test_pred, alpha=0.5)
    # plt.xlabel('Real Values')
    # plt.ylabel('Predicted Values')
    # plt.title(f'Comparison of Real and Predicted Values for {model_name}')
    # plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Diagonal line
    # plt.savefig(f'predicted_vs_real_{model_name}_v1.png')
    # plt.show()
    
    
    # # Calcul des résidus
    # residuals = y_test - y_test_pred

    # # Tracer le graphique des résidus
    # plt.figure(figsize=(10, 6))
    # plt.scatter(y_test_pred, residuals, alpha=0.5)
    # plt.xlabel('Predicted Values')
    # plt.ylabel('Residuals')
    # plt.title(f'Residual Plot for {model_name}')
    # plt.axhline(y=0, color='k', linestyle='--')  # Ligne horizontale à zéro
    # plt.savefig(f'residual_plot_{model_name}_v1.png')
    # plt.show()
    
    # # Tracer l'histogramme des résidus
    # plt.figure(figsize=(10, 6))
    # plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
    # plt.xlabel('Residuals')
    # plt.ylabel('Frequency')
    # plt.title(f'Histogram of Residuals for {model_name}')
    # plt.savefig(f'residuals_histogram_{model_name}_v1.png')
    # plt.show()

    # # Tracer le tracé KDE des résidus
    # plt.figure(figsize=(10, 6))
    # sns.kdeplot(residuals, shade=True)
    # plt.xlabel('Residuals')
    # plt.ylabel('Density')
    # plt.title(f'KDE Plot of Residuals for {model_name}')
    # plt.savefig(f'residuals_kde_{model_name}_v1.png')
    # plt.show()


    # Enregistrement des résultats dans le DataFrame
    global results_df
    new_row = pd.DataFrame([{
        'Model Name': model_name,
        'Best Parameters': best_params,
        'MSE Train': mse_train,
        'MSE Test': mse_test,
        'R2 Train': r2_train,
        'R2 Test': r2_test,
        'Best CV R2 Test': best_cv_score,
        'Fit Time': fit_time
    }])
    results_df = pd.concat([results_df, new_row], ignore_index=True)


2024/04/24 07:40:11 INFO mlflow.tracking.fluent: Experiment with name 'Gradient_Boosting_gas_k_val' does not exist. Creating a new experiment.


In [15]:
for k in range(15, 30):
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.3,
                                                        random_state=42)

    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

    pipe_trans_num = Pipeline(steps=[('scaler', StandardScaler())])
    pipe_trans_cat = Pipeline(steps=[('ohe', OneHotEncoder())])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', pipe_trans_num, numeric_features),
            ('cat', pipe_trans_cat, categorical_features)
        ])

    k_val = k

    model_pipelines = {
        f"GradientBoosting_k_{k}": Pipeline([("preprocessor", preprocessor), ("imputer", KNNImputer(n_neighbors=k_val)), ("model", GradientBoostingRegressor())])
    }
    
    param_grids = {
        f"GradientBoosting_k_{k}": {'model__n_estimators': [150], 'model__learning_rate': [0.1], 
                            'model__max_depth': [2], 
                            'model__min_samples_split': [20], 'model__min_samples_leaf':[10], 
                            'model__max_features': [.75]}
    }

    # Recherche des meilleurs hyperparamètres et enregistrement du meilleur score de validation croisée
    best_params = {}
    best_cv_scores = {}

    for model_name, param_grid in param_grids.items():
        model = model_pipelines[model_name]

        if param_grid:
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', verbose=1)
            grid_search.fit(X_train, y_train)
            best_params[model_name] = grid_search.best_params_
            best_cv_scores[model_name] = max(grid_search.cv_results_['mean_test_score'])
        else:
            best_params[model_name] = None
            best_cv_scores[model_name] = None

    # Entraînement, évaluation et enregistrement des meilleurs modèles
    for model_name, params in best_params.items():
        model = model_pipelines[model_name]

        train_eval_model(model, X_train, y_train, X_test, y_test, model_name, params, best_cv_scores[model_name])



Fitting 5 folds for each of 1 candidates, totalling 5 fits


GradientBoosting_k_15 - MSE Train: 0.6834824816511306, MSE Test: 0.7981002693418268, R2 Train: 0.6811806323787871, R2 Test: 0.5582439327481552
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/var/folders/17/7v9_fz593_xfss9zy0542f940000gn/T/ipykernel_71726/3624885854.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


GradientBoosting_k_16 - MSE Train: 0.6729435750214366, MSE Test: 0.7990354729881791, R2 Train: 0.6860966435967792, R2 Test: 0.5577262886616134
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_k_17 - MSE Train: 0.6814858283654718, MSE Test: 0.8038232242172099, R2 Train: 0.682111997490523, R2 Test: 0.5550762229553821
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_k_18 - MSE Train: 0.6798255025980511, MSE Test: 0.7995919901054755, R2 Train: 0.6828864782203516, R2 Test: 0.5574182511598871
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_k_19 - MSE Train: 0.6843550481637628, MSE Test: 0.8055986981084704, R2 Train: 0.6807736122850273, R2 Test: 0.5540934813202258
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_k_20 - MSE Train: 0.6738666432157339, MSE Test: 0.8201944106623437, R2 Train: 0.6856660663312637, R2 Test: 0.5460146160144232
Fitting 5 folds for each of 1 candidates, totallin

In [16]:
results_df.head(50)

,Model Name,Best Parameters,MSE Train,MSE Test,R2 Train,R2 Test,Best CV R2 Test,Fit Time
0,GradientBoosting_k_15,"{'model__learning_rate': 0.1, 'model__max_dept...",0.683482,0.798100,0.681181,0.558244,0.586806,0.338951
1,GradientBoosting_k_16,"{'model__learning_rate': 0.1, 'model__max_dept...",0.672944,0.799035,0.686097,0.557726,0.587474,0.333740
2,GradientBoosting_k_17,"{'model__learning_rate': 0.1, 'model__max_dept...",0.681486,0.803823,0.682112,0.555076,0.591715,0.420729
3,GradientBoosting_k_18,"{'model__learning_rate': 0.1, 'model__max_dept...",0.679826,0.799592,0.682886,0.557418,0.586688,0.383327
4,GradientBoosting_k_19,"{'model__learning_rate': 0.1, 'model__max_dept...",0.684355,0.805599,0.680774,0.554093,0.588814,0.473636
5,GradientBoosting_k_20,"{'model__learning_rate': 0.1, 'model__max_dept...",0.673867,0.820194,0.685666,0.546015,0.584067,0.389372
6,GradientBoosting_k_21,"{'model__learning_rate': 0.1, 'model__max_dept...",0.681941,0.802238,0.681900,0.555953,0.590401,0.343287
7,GradientBoosting_k_22,"{'model__learning_rate': 0.1, 'model__max_dept...",0.681636,0.800281,0.682042,0.557037,0.586131,0.337820
8,GradientBoosting_k_23,"{'model__learning_rate': 0.1, 'model__max_dept...",0.678754,0.801535,0.683386,0.556343,0.588064,0.306967
9,GradientBoosting_k_24,"{'model__learning_rate': 0.1, 'model__max_dept...",0.666965,0.792118,0.688885,0.561555,0.588673,0.365021


In [17]:
# Retrieve the current MLflow experiment name
mlflow_experiment_name = mlflow.get_experiment_by_name('Gradient_Boosting_gas_k_val').name

# Construct a filename using the experiment name
filename = f"{mlflow_experiment_name}_results.csv"

# Save the DataFrame
results_df.to_csv(filename, index=False)
print(f"Results saved as {filename}")

Results saved as Gradient_Boosting_gas_k_val_results.csv


## 4. Models for different train_test_split

In [18]:
# Configurer MLflow
mlflow.set_experiment('Gradient_Boosting_gas_train_test')

# Initialiser le DataFrame des résultats
results_df = pd.DataFrame(columns=['Model Name', 'Best Parameters', 'MSE Train', 'MSE Test', 'R2 Train', 'R2 Test', 'Best CV R2 Test', 'Fit Time'])

def train_eval_model(model, X_train, y_train, X_test, y_test, model_name, best_params=None, best_cv_score=None):
    # Set model parameters if provided
    if best_params:
        model.set_params(**best_params)

    # Mesurer le temps de début
    start_time = time.time()
    # Entraîner le modèle
    model.fit(X_train, y_train)
    # Mesurer le temps de fin et calculer le temps de fitting
    fit_time = time.time() - start_time

    # Prédictions sur les données d'entraînement et de test
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcul des métriques de performance
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    # Enregistrement de la performance du modèle avec MLflow
    with mlflow.start_run(run_name=f"{model_name}"):
        if best_params:
            mlflow.log_params(best_params)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("mse_test", mse_test)
        mlflow.log_metric("r2_train", r2_train)
        mlflow.log_metric("r2_test", r2_test)
        if best_cv_score is not None:
            mlflow.log_metric("best_cv_r2_test", best_cv_score)
        mlflow.sklearn.log_model(model, f"model_{model_name}")

        print(f"{model_name} - MSE Train: {mse_train}, MSE Test: {mse_test}, R2 Train: {r2_train}, R2 Test: {r2_test}")

    # # Plotting the predicted vs real values
    # plt.figure(figsize=(10, 6))
    # plt.scatter(y_test, y_test_pred, alpha=0.5)
    # plt.xlabel('Real Values')
    # plt.ylabel('Predicted Values')
    # plt.title(f'Comparison of Real and Predicted Values for {model_name}')
    # plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Diagonal line
    # plt.savefig(f'predicted_vs_real_{model_name}_v1.png')
    # plt.show()
    
    
    # # Calcul des résidus
    # residuals = y_test - y_test_pred

    # # Tracer le graphique des résidus
    # plt.figure(figsize=(10, 6))
    # plt.scatter(y_test_pred, residuals, alpha=0.5)
    # plt.xlabel('Predicted Values')
    # plt.ylabel('Residuals')
    # plt.title(f'Residual Plot for {model_name}')
    # plt.axhline(y=0, color='k', linestyle='--')  # Ligne horizontale à zéro
    # plt.savefig(f'residual_plot_{model_name}_v1.png')
    # plt.show()
    
    # # Tracer l'histogramme des résidus
    # plt.figure(figsize=(10, 6))
    # plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
    # plt.xlabel('Residuals')
    # plt.ylabel('Frequency')
    # plt.title(f'Histogram of Residuals for {model_name}')
    # plt.savefig(f'residuals_histogram_{model_name}_v1.png')
    # plt.show()

    # # Tracer le tracé KDE des résidus
    # plt.figure(figsize=(10, 6))
    # sns.kdeplot(residuals, shade=True)
    # plt.xlabel('Residuals')
    # plt.ylabel('Density')
    # plt.title(f'KDE Plot of Residuals for {model_name}')
    # plt.savefig(f'residuals_kde_{model_name}_v1.png')
    # plt.show()


    # Enregistrement des résultats dans le DataFrame
    global results_df
    new_row = pd.DataFrame([{
        'Model Name': model_name,
        'Best Parameters': best_params,
        'MSE Train': mse_train,
        'MSE Test': mse_test,
        'R2 Train': r2_train,
        'R2 Test': r2_test,
        'Best CV R2 Test': best_cv_score,
        'Fit Time': fit_time
    }])
    results_df = pd.concat([results_df, new_row], ignore_index=True)


2024/04/24 07:41:46 INFO mlflow.tracking.fluent: Experiment with name 'Gradient_Boosting_gas_train_test' does not exist. Creating a new experiment.


In [19]:
for k in [23]:
    for i in range(50):
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_size=0.3,
                                                            random_state=i
                                                            )

        numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
        categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

        pipe_trans_num = Pipeline(steps=[('scaler', StandardScaler())])
        pipe_trans_cat = Pipeline(steps=[('ohe', OneHotEncoder())])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', pipe_trans_num, numeric_features),
                ('cat', pipe_trans_cat, categorical_features)
            ])

        k_val = k

        model_pipelines = {
            f"GradientBoosting_train_test_k_{k_val}_{i}": Pipeline([("preprocessor", preprocessor), ("imputer", KNNImputer(n_neighbors=k_val)), ("model", GradientBoostingRegressor())])
        }
        
        param_grids = {
            f"GradientBoosting_train_test_k_{k_val}_{i}": {'model__n_estimators': [150], 'model__learning_rate': [0.1], 
                            'model__max_depth': [2], 
                            'model__min_samples_split': [20], 'model__min_samples_leaf':[10], 
                            'model__max_features': [.75]}
                    }

        # Recherche des meilleurs hyperparamètres et enregistrement du meilleur score de validation croisée
        best_params = {}
        best_cv_scores = {}

        for model_name, param_grid in param_grids.items():
            model = model_pipelines[model_name]

            if param_grid:
                grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', verbose=1)
                grid_search.fit(X_train, y_train)
                best_params[model_name] = grid_search.best_params_
                best_cv_scores[model_name] = max(grid_search.cv_results_['mean_test_score'])
            else:
                best_params[model_name] = None
                best_cv_scores[model_name] = None

        # Entraînement, évaluation et enregistrement des meilleurs modèles
        for model_name, params in best_params.items():
            model = model_pipelines[model_name]

            train_eval_model(model, X_train, y_train, X_test, y_test, model_name, params, best_cv_scores[model_name])



Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_train_test_k_23_0 - MSE Train: 0.6395370283887855, MSE Test: 0.8964912045363566, R2 Train: 0.6879911384132007, R2 Test: 0.5607744127159113
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/var/folders/17/7v9_fz593_xfss9zy0542f940000gn/T/ipykernel_71726/1272157839.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


GradientBoosting_train_test_k_23_1 - MSE Train: 0.6590806386577924, MSE Test: 0.8364218072397377, R2 Train: 0.688759466809759, R2 Test: 0.5555098588016929
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_train_test_k_23_2 - MSE Train: 0.6184083293330039, MSE Test: 0.9225775286025591, R2 Train: 0.6931137678050587, R2 Test: 0.5641877503063922
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_train_test_k_23_3 - MSE Train: 0.6433941305917862, MSE Test: 0.9237822366623587, R2 Train: 0.6912775732282397, R2 Test: 0.5281623525152562
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_train_test_k_23_4 - MSE Train: 0.6788017687812768, MSE Test: 0.7932365245789484, R2 Train: 0.6767703876166014, R2 Test: 0.5873835236812004
Fitting 5 folds for each of 1 candidates, totalling 5 fits
GradientBoosting_train_test_k_23_5 - MSE Train: 0.6681099954738872, MSE Test: 0.8797605727370393, R2 Train: 0.6768063326523628, R2 Test: 0.56

In [20]:
results_df.head(50)

,Model Name,Best Parameters,MSE Train,MSE Test,R2 Train,R2 Test,Best CV R2 Test,Fit Time
0,GradientBoosting_train_test_k_23_0,"{'model__learning_rate': 0.1, 'model__max_dept...",0.639537,0.896491,0.687991,0.560774,0.575274,0.418869
1,GradientBoosting_train_test_k_23_1,"{'model__learning_rate': 0.1, 'model__max_dept...",0.659081,0.836422,0.688759,0.555510,0.555014,0.277961
2,GradientBoosting_train_test_k_23_2,"{'model__learning_rate': 0.1, 'model__max_dept...",0.618408,0.922578,0.693114,0.564188,0.583509,0.362036
3,GradientBoosting_train_test_k_23_3,"{'model__learning_rate': 0.1, 'model__max_dept...",0.643394,0.923782,0.691278,0.528162,0.597441,0.317119
4,GradientBoosting_train_test_k_23_4,"{'model__learning_rate': 0.1, 'model__max_dept...",0.678802,0.793237,0.676770,0.587384,0.564824,0.478928
5,GradientBoosting_train_test_k_23_5,"{'model__learning_rate': 0.1, 'model__max_dept...",0.668110,0.879761,0.676806,0.560146,0.580440,0.307230
6,GradientBoosting_train_test_k_23_6,"{'model__learning_rate': 0.1, 'model__max_dept...",0.638817,0.901269,0.684150,0.570722,0.563849,0.290400
7,GradientBoosting_train_test_k_23_7,"{'model__learning_rate': 0.1, 'model__max_dept...",0.623227,0.936168,0.686692,0.567563,0.576697,0.408403
8,GradientBoosting_train_test_k_23_8,"{'model__learning_rate': 0.1, 'model__max_dept...",0.666752,0.863259,0.666296,0.598562,0.551535,0.488847
9,GradientBoosting_train_test_k_23_9,"{'model__learning_rate': 0.1, 'model__max_dept...",0.625550,0.917707,0.687901,0.572550,0.580023,0.331840


In [21]:
# Retrieve the current MLflow experiment name
mlflow_experiment_name = mlflow.get_experiment_by_name('Gradient_Boosting_gas_train_test').name

# Construct a filename using the experiment name
filename = f"{mlflow_experiment_name}_results.csv"

# Save the DataFrame
results_df.to_csv(filename, index=False)
print(f"Results saved as {filename}")

Results saved as Gradient_Boosting_gas_train_test_results.csv


In [22]:
# !mlflow ui